# Compare with `abess`

In [1]:
import time
import numpy as np
import pandas as pd
from IPython.display import display

import jax.numpy as jnp
from skscope import ScopeSolver


from abess.datasets import make_glm_data
from abess import LinearRegression, LogisticRegression

In [2]:
iterables = [['Linear', 'Non-Neg', 'Logistic'], ['abess', 'skscope']]
index = pd.MultiIndex.from_product(iterables, names=['Task', 'Model'])
res = pd.DataFrame(columns=['Accuracy', 'Time'], index = index)

In [3]:
rng = np.random.default_rng()
rng.choice(np.arange(1, 4), size=5) * rng.choice([1], size=5)

array([3, 3, 1, 3, 2])

In [4]:
def test_time(n=500, p=1000, s=5, random_state=None):
    print('='*20 + f'  n={n}, p={p}, s={s}  ' + '='*20 )
    rng = np.random.default_rng(random_state)
    true_support_set = rng.choice(np.arange(p), size=s, replace=False)
    real_coef = np.zeros(p)

    iterables = [['Linear', 'Non-Neg', 'Logistic'], ['abess', 'skscope']]
    index = pd.MultiIndex.from_product(iterables, names=['Task', 'Model'])
    res = pd.DataFrame(columns=['Accuracy', 'Time'], index = index)

    for type in ['Linear', 'Non-Neg', 'Logistic']:
        if type == 'Linear':
            real_coef[true_support_set] = rng.choice(np.arange(1, 4), size=s) * rng.choice([1, -1], size=s)
            data = make_glm_data(n=n, p=p, k=s, family='gaussian', coef_=real_coef)
            model = LinearRegression(support_size=s)
            def objective(params):
                loss = jnp.mean((y - X @ params) ** 2)
                return loss
            solver = ScopeSolver(p, sparsity=s)
        elif type == 'Non-Neg':
            real_coef[true_support_set] = rng.choice(np.arange(1, 4), size=s) * rng.choice([1], size=s)
            data = make_glm_data(n=n, p=p, k=s, family='gaussian', coef_=real_coef)
            model = LinearRegression(support_size=s)
            def objective(params):
                loss = jnp.mean((y - X @ params) ** 2)
                return loss
            solver = ScopeSolver(p, sparsity=s)
        elif type == 'Logistic':
            real_coef[true_support_set] = rng.choice(np.arange(1, 4), size=s) * rng.choice([1, -1], size=s)
            data = make_glm_data(n=n, p=p, k=s, family='binomial', coef_=real_coef)
            model = LogisticRegression(support_size=s)
            def objective(params):
                xbeta = jnp.clip(X @ params, -30, 30)
                return jnp.mean(jnp.log(1 + jnp.exp(xbeta)) - y * xbeta)
            solver = ScopeSolver(p, sparsity=s)
    
        X, y = data.x, data.y
        
        # abess
        t_begin = time.time()
        model.fit(X, y)
        t_abess = time.time() - t_begin
        acc_abess = len(set(np.nonzero(model.coef_)[0]) & set(true_support_set)) / s
        res.loc[(type, 'abess')] = [acc_abess, np.round(t_abess, 4)]
        
        # skscope
        t_begin = time.time()
        params = solver.solve(objective, jit=True)
        t_skscope = time.time() - t_begin
        acc_skscope = len(set(np.nonzero(params)[0]) & set(np.nonzero(data.coef_)[0])) / s
        res.loc[(type, 'skscope')] = [acc_skscope, np.round(t_skscope, 4)]

    display(res)


In [5]:
settings = [
    (500, 1000, 5),
    (2000, 5000, 10),
    (5000, 10000, 10),
]
for setting in settings:
    n, p, s = setting
    test_time(n=n, p=p, s=s)

====================  n=500, p=1000, s=5  ====================


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Accuracy    Time
Task     Model                   
Linear   abess        1.0  0.0142
         skscope      1.0  0.5137
Non-Neg  abess        1.0  0.0066
         skscope      1.0  0.3685
Logistic abess        1.0  0.0101
         skscope      1.0  0.4023

====================  n=2000, p=5000, s=10  ====================


Accuracy    Time
Task     Model                   
Linear   abess        1.0  0.2543
         skscope      1.0  1.7219
Non-Neg  abess        1.0  0.2544
         skscope      1.0  1.6639
Logistic abess        1.0  0.2788
         skscope      1.0   1.801

====================  n=5000, p=10000, s=10  ====================


Accuracy    Time
Task     Model                   
Linear   abess        1.0  1.2848
         skscope      1.0  6.6585
Non-Neg  abess        1.0  1.2518
         skscope      1.0  6.7542
Logistic abess        1.0  1.3137
         skscope      1.0  7.7955